# Курс ["Алгоритмы анализа данных"](https://gb.ru/lessons/177362)

# Урок 8. Снижение размерности данных

## Задание 1.

### Можно ли отобрать наиболее значимые признаки с помощью PCA?

Очень опосредованно. Обычно его так не используют (ИМХО), но при очень большом желании можно провести реверс-инженеринг и посмотреть наиболее коррелируемые вектора исходного набора и получившегося в результате метода главных компонент. 

## Задание 2. (*)

### Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [1]:
import numpy as np
from sklearn.datasets import load_iris

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import warnings
warnings.simplefilter('ignore')

In [2]:
X, y = load_iris(return_X_y=True)

In [3]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [4]:
U, D, V = np.linalg.svd(X_scaled)
D

array([20.92306556, 11.7091661 ,  4.69185798,  1.76273239])

In [5]:
eig_sum = sum(D)
var_exp = [(i / eig_sum) * 100 for i in D]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описвыаемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описвыаемая каждой из компонент 
[53.529717882362405, 29.956812784020087, 12.003682417209484, 4.509786916408035]
Кумулятивная доля дисперсии по компонентам 
[ 53.52971788  83.48653067  95.49021308 100.        ]


Мы потеряем 5% информации, оставив три признака. Если эти 5% не определяют размер нашей премии, то:

In [6]:
W = V[:, :3]
W

array([[ 0.52106591, -0.26934744,  0.5804131 ],
       [-0.37741762, -0.92329566, -0.02449161],
       [ 0.71956635, -0.24438178, -0.14212637],
       [ 0.26128628, -0.12350962, -0.80144925]])

In [8]:
X_ = X @ W

## Задание 3. (*)

### Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

In [10]:
# Загрузка данных, разбиение и стандартизация
X, y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.3, 
                                                    random_state = 5)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
#  Обучение модели до применения PCA

lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
print(accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

0.9809523809523809 0.9777777777777777


In [12]:
# Обучение модели после применения PCA

X_train, X_test, y_train, y_test = train_test_split(X_, y, 
                                                    test_size = 0.3, 
                                                    random_state = 5)
lr.fit(X_train, y_train)
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
print(accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

0.9714285714285714 0.9555555555555556


Возможно, мы уменьшили переобучение.. Но результаты очень похожи